In [2]:
import pandas as pd
import numpy as np
import os

from collections import Counter

import chromadb
from chromadb.utils.embedding_functions import OpenCLIPEmbeddingFunction
from chromadb.utils.data_loaders import ImageLoader

In [29]:
# data_train = pd.read_csv("datasets/train.csv")

# Filter to only include item_weight entity

# Extract numeric value and unit from entity_value
# def split_value(value):
#     value = value.strip()
#     parts = value.strip().split(' ')
#     if len(parts) == 2:
#         try: 
#             float(parts[0])
#         except:
#             return None, None
#         return float(parts[0]), parts[1]
#     return None, None

# data_train[['numeric_value', 'unit']] = data_train['entity_value'].apply(split_value).apply(pd.Series)


# # Drop rows with missing values
# data_train.dropna(inplace=True)

# sampled_data_train = data_train.groupby('unit').apply(lambda x: x.sample(min(len(x), 100))).reset_index(drop=True)
# # Save processed data to a new CSV file
# # sampled_data_train.to_csv('vector.csv', index=True)
sampled_data_train = pd.read_csv('vector.csv')

# print("Data preprocessing complete, dataset ready to add to vectorDB")

Data preprocessing complete, dataset ready to add to vectorDB


/tmp/ipykernel_422593/2178744517.py:23: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_data_train = data_train.groupby('unit').apply(lambda x: x.sample(min(len(x), 100))).reset_index(drop=True)


In [30]:
# data_test = pd.read_csv("datasets/train.csv")

# # Filter to only include item_weight entity

# # Extract numeric value and unit from entity_value
# def split_value(value):
#     value = value.strip()
#     parts = value.strip().split(' ')
#     if len(parts) == 2:
#         try: 
#             float(parts[0])
#         except:
#             return None, None
#         return float(parts[0]), parts[1]
#     return None, None

# data_test[['numeric_value', 'unit']] = data_test['entity_value'].apply(split_value).apply(pd.Series)


# # Drop rows with missing values
# data_test.dropna(inplace=True)

# sampled_data_test = data_test.groupby('entity_name').apply(lambda x: x.sample(min(len(x), 1000))).reset_index(drop=True)
# # Save processed data to a new CSV file
# # sampled_data_test.to_csv('test.csv', index=True)


# print("Data preprocessing complete, dataset ready to add to vectorDB")

Data preprocessing complete, dataset ready to add to vectorDB


/tmp/ipykernel_422593/2735091366.py:23: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_data_test = data_test.groupby('entity_name').apply(lambda x: x.sample(min(len(x), 1000))).reset_index(drop=True)


In [33]:
print(len(sampled_data_train), len(sampled_data_test))

2343 8000


In [34]:
from src.training.height.utils_height import download_images
download_images(sampled_data_test['image_link'].tolist(), sampled_data_test.index.tolist(), "datasets/test_images")
download_images(sampled_data_train['image_link'].tolist(), sampled_data_train.index.tolist(), "datasets/vector_images")

100%|██████████| 2343/2343 [01:55<00:00, 20.36it/s]


In [12]:
client = chromadb.Client()
embedding_function = OpenCLIPEmbeddingFunction()
data_loader = ImageLoader()

/home/blank/miniconda3/envs/tf/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [35]:
collection = client.get_or_create_collection(
    name="products",
    embedding_function=embedding_function,
    data_loader=data_loader
)

In [36]:
vector_data = []
for idx, row in sampled_data_train.iterrows():
    image_location = os.path.join("datasets/vector_images", f"{idx}.jpg")
    metadata = {
        "unit": row["unit"],
        "entity_name": row["entity_name"],
        "numeric_value": row["numeric_value"]
    }
    vector_data.append({
        "image_location": image_location,
        "metadata": metadata
    })

In [37]:
vector_data[1]

{'image_location': 'datasets/vector_images/1.jpg',
 'metadata': {'unit': 'candela',
  'entity_name': 'item_weight',
  'numeric_value': 1.0}}

In [38]:
for idx, val in enumerate(vector_data):
    if idx % 100 == 0 and idx != 0:
        print("At", idx)
    collection.add(
        ids = [str(idx)],
        uris=[val['image_location']],
        metadatas=[val['metadata']]
    )

At 100
At 200
At 300
At 400
At 500
At 600
At 700
At 800
At 900
At 1000
At 1100
At 1200
At 1300
At 1400
At 1500
At 1600
At 1700
At 1800
At 1900
At 2000
At 2100
At 2200
At 2300


In [51]:
def query_and_find_most_frequent(img_path, entity_name):
    # query_text = remove_numbers(query_text)
    results = collection.query(query_uris=[img_path],
                               where={"entity_name": entity_name},
                               n_results=13)
    # print(results) 
    string_values = [result['unit'] for result in results['metadatas'][0]]
    
    string_counts = Counter(string_values)
    # print(string_values)
    # print(string_counts.most_common(1)) 
    most_frequent_string = string_counts.most_common(1)[0][0]
    
    return most_frequent_string

In [24]:
query_data = pd.read_csv("test.csv")
query_data_id = query_data["id"]
query_data_unit = query_data["unit"]
query_data_entity = query_data["entity_name"]

In [54]:
def process_item(id, label, entity_name, idx):
    path = "datasets/test_images" + str(id) + ".jpg"
    res = query_and_find_most_frequent(path, entity_name)
    return idx, res == label

# Initialize variables
total_count = len(query_data_id)
count = 0
wrong_values = []
len_item = 0
# Process each item in a single thread
for idx, (label, id, entity) in enumerate(zip(query_data_unit, query_data_id, query_data_entity)):
    if entity == "height" or  entity == "depth" or entity == "width":
        len_item += 1
        continue
    else:
        path = "datasets/test_images/" + str(id) + ".jpg"
        res = query_and_find_most_frequent(path, entity)
        if idx % 500 == 0 and idx != 0:
            print("At", idx, "Accuracy:", str(count / ((idx+1) - len_item)))
        if res == label:
            count += 1
        else: 
            wrong_values.append((id, label, res))

# Print the final accuracy
print(count / total_count)

At 2000 Accuracy: 0.0
At 2500 Accuracy: 0.18562874251497005
At 3000 Accuracy: 0.1918081918081918
At 3500 Accuracy: 0.18654230512991338
At 4000 Accuracy: 0.1809095452273863
At 4500 Accuracy: 0.17712914834066373
At 5000 Accuracy: 0.17960679773408864
At 5500 Accuracy: 0.2967723507569266
At 6000 Accuracy: 0.38465383654086477
At 6500 Accuracy: 0.4267940457676072
0.286


In [53]:
print(wrong_values)

[(2000, 'millilitre', 'gallon'), (2001, 'millilitre', 'cup'), (2002, 'litre', 'millilitre'), (2003, 'millilitre', 'gallon'), (2004, 'cup', 'pint'), (2005, 'millilitre', 'pint'), (2008, 'litre', 'millilitre'), (2009, 'ounce', 'pint'), (2010, 'millilitre', 'quart'), (2011, 'millilitre', 'cup'), (2012, 'millilitre', 'pint'), (2013, 'millilitre', 'quart'), (2014, 'litre', 'decilitre'), (2015, 'millilitre', 'pint'), (2017, 'millilitre', 'gallon'), (2018, 'ounce', 'cup'), (2019, 'millilitre', 'quart'), (2020, 'millilitre', 'litre'), (2021, 'millilitre', 'quart'), (2022, 'millilitre', 'quart'), (2023, 'millilitre', 'litre'), (2025, 'millilitre', 'gallon'), (2026, 'millilitre', 'quart'), (2027, 'millilitre', 'quart'), (2028, 'millilitre', 'cup'), (2029, 'cup', 'gallon'), (2031, 'cup', 'millilitre'), (2033, 'decilitre', 'pint'), (2035, 'litre', 'millilitre'), (2036, 'millilitre', 'centilitre'), (2037, 'millilitre', 'gallon'), (2038, 'millilitre', 'litre'), (2040, 'millilitre', 'gallon'), (2041,